# **Combine building density with training data**

In [22]:
import pandas as pd
from shapely.geometry import Point, Polygon
from shapely.wkt import loads
import geopandas as gpd

In [23]:
# Load the CSV file into a GeoDataFrame
def load_grid_data(csv_file):
    # Read the CSV file
    grid_data = pd.read_csv(csv_file)
    
    # Convert WKT geometries to shapely objects
    grid_data['geometry'] = grid_data['geometry_wkt'].apply(lambda wkt: loads(wkt))
    
    # Create a GeoDataFrame
    grid_gdf = gpd.GeoDataFrame(grid_data, geometry='geometry', crs="EPSG:4326")
    return grid_gdf

In [24]:
# Function to locate the density for a random point
def get_density_for_point(grid_gdf, point_lon, point_lat):
    # Create a point geometry
    point = Point(point_lon, point_lat)
    
    # Find the grid cell containing the point
    matching_cell = grid_gdf[grid_gdf.geometry.contains(point)]
    
    if not matching_cell.empty:
        # Extract the density value
        density = matching_cell['density'].iloc[0]
        return density
    else:
        return None  # Point is outside the grid

In [25]:
# Function to locate the density for a point
def get_density_for_point(grid_gdf, point_lon, point_lat):
    # Create a point geometry
    point = Point(point_lon, point_lat)
    
    # Find the grid cell containing the point
    matching_cell = grid_gdf[grid_gdf.geometry.contains(point)]
    
    if not matching_cell.empty:
        # Extract the density value
        density = matching_cell['density'].iloc[0]
        return density
    else:
        return None  # Point is outside the grid

In [26]:
# Add density feature to a DataFrame
def add_density_to_dataframe(points_df, grid_gdf):
    # Create a new 'density' column by applying the function to each row
    points_df['density'] = points_df.apply(
        lambda row: get_density_for_point(grid_gdf, row['Longitude'], row['Latitude']),
        axis=1
    )
    return points_df

In [27]:
# Load the training data from csv file and display the first few rows to inspect the data
ground_df = pd.read_csv("/kaggle/input/training-data/Training_data_uhi_index_UHI2025-v2.csv")
ground_df.head()

,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24-07-2021 15:53,1.030289
1,-73.909187,40.813045,24-07-2021 15:53,1.030289
2,-73.909215,40.812978,24-07-2021 15:53,1.023798
3,-73.909242,40.812908,24-07-2021 15:53,1.023798
4,-73.909257,40.812845,24-07-2021 15:53,1.021634


In [28]:
# Load the grid data
grid_csv = '/kaggle/input/grid-density/grid_density.csv'  
grid_gdf = load_grid_data(grid_csv)

In [29]:
# Add density to the points DataFrame
final_df = add_density_to_dataframe(ground_df, grid_gdf)

# Display the resulting DataFrame
print(final_df)

       Longitude   Latitude          datetime  UHI Index  density
0     -73.909167  40.813107  24-07-2021 15:53   1.030289        8
1     -73.909187  40.813045  24-07-2021 15:53   1.030289        8
2     -73.909215  40.812978  24-07-2021 15:53   1.023798        8
3     -73.909242  40.812908  24-07-2021 15:53   1.023798        8
4     -73.909257  40.812845  24-07-2021 15:53   1.021634        8
...          ...        ...               ...        ...      ...
11224 -73.957050  40.790333  24-07-2021 15:57   0.972470        0
11225 -73.957063  40.790308  24-07-2021 15:57   0.972470        0
11226 -73.957093  40.790270  24-07-2021 15:57   0.981124        0
11227 -73.957112  40.790253  24-07-2021 15:59   0.981245        0
11228 -73.957128  40.790237  24-07-2021 15:59   0.983408        0

[11229 rows x 5 columns]


In [30]:
# Save the DataFrame to a CSV file
output_csv = "training_data_with_density.csv"
final_df.to_csv(output_csv, index=False)

print(f"The DataFrame has been saved to {output_csv}")

The DataFrame has been saved to training_data_with_density.csv
